# Data Report

We download 2 main datasets for our analysis:

1.
Quarterly Workforce Indicators (QWI) from the U.S. Census Bureau. This dataset provides detailed local estimates of various employment and earnings indicators. Employment, earnings, gross job creation and destruction, and worker turnover are available at different levels of geography.

We have downloaded this dataset from https://ledextract.ces.census.gov/qwi/all website.  we requested a csv file containing the following variables: geography (50 states), firm’s industry (NAICS sectors), firm size (measured in number of employees), worker’s sex, worker’s education level, and our dependent variable, earnings. The data is quarterly measured, and we request the data from 2012Q1 to 2021Q4, with a total of 10 years.
Also, we used a dataset from class to match the States name with States' geography number (https://datascience.quantecon.org/assets/data/cb_2016_us_state_5m.zip).

**Key Variables:**

geography: Geography numbers represent States in the U.S. (https://lehd.ces.census.gov/data/schema/latest/label_fipsnum.csv)

industry: Industries' numbers represent industries

year: From 2012 to 2021

quarters: Quarter1 to Quarter4

sex: 1 for male, 2 for female

education: 
 E0-All Education Categories
 E1-Less than high school
 E2-High school or equivalent, no college
 E3-Some college or Associate degree
 E4-Bachelor's degree or advanced degree
 E5-Educational attainment not available (workers aged 24 or younger)

firmsize:
 0 All Firm Sizes
 1 0-19 Employees
 2 20-49 Employees
 3 50-249 Employees
 4 250-499 Employees
 5 500+ Employees

EarnS: Average monthly earnings of employees with stable jobs(i.e., worked with the same firm throughout the quarter)

NAME: States' name

geometry: longitude and latitude

notes: We dropped several variables to make our data easy to read and analyze.

2.
Post-Secondary Employment Outcomes (PSEO). PSEO are experimental tabulations developed by researchers at the U.S. Census Bureau. PSEO data provide earnings and employment outcomes for college and university graduates by degree level, degree major, and post-secondary institution. This dataset is downloaded from website https://lehd.ces.census.gov/data/pseo_experimental.html.

We focus on the PESO data of New York in particular. We want to analyze the relationship between degree majors of college graduates and the industries they chose to work in.

**Key Variables**: 

label_cipcode: college students' major

label_industry: industries' name

The code below reads and cleans the data for our analysis:


In [90]:
#Import Packages 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import geopandas as gpd
from shapely.geometry import Point
from pyecharts.charts import Geo
from pyecharts import options as opts
from pyecharts.charts import *
from pyecharts.components import Table
from pyecharts.commons.utils import JsCode
from pyecharts.globals import ThemeType,SymbolType,ChartType
from pyecharts.datasets import register_url
import json
from pyecharts.faker import Faker
import itertools

## Cleaning the first dataset: QWI

In [2]:
# Read in data 
df_earnings=pd.read_csv('QWI2.csv') #Read Quarterly Workforce Indicators (QWI)
label_education = pd.read_csv('label_education.csv')

In [3]:
df_earnings.head()

,periodicity,seasonadj,geo_level,geography,ind_level,industry,ownercode,sex,agegrp,race,...,EarnHirAS,EarnHirNS,EarnS,EarnSepS,sEarnBeg,sEarnHirAS,sEarnHirNS,sEarnS,sEarnSepS,version
0,Q,U,S,30,S,11,A05,1,A00,A0,...,2069.0,2053.0,2268.0,1888.0,1,1,1,1,1,QWI_FS MT 30 1993:1-2021:4 V4.9.0 R2022Q4 qwip...
1,Q,U,S,30,S,11,A05,1,A00,A0,...,2033.0,2007.0,2486.0,1913.0,1,1,1,1,1,QWI_FS MT 30 1993:1-2021:4 V4.9.0 R2022Q4 qwip...
2,Q,U,S,30,S,11,A05,1,A00,A0,...,1785.0,1727.0,2605.0,1878.0,1,1,1,1,1,QWI_FS MT 30 1993:1-2021:4 V4.9.0 R2022Q4 qwip...
3,Q,U,S,30,S,11,A05,1,A00,A0,...,2079.0,2140.0,2734.0,1572.0,1,1,1,1,1,QWI_FS MT 30 1993:1-2021:4 V4.9.0 R2022Q4 qwip...
4,Q,U,S,30,S,11,A05,1,A00,A0,...,1244.0,1407.0,1412.0,1110.0,1,1,1,1,1,QWI_FS MT 30 1993:1-2021:4 V4.9.0 R2022Q4 qwip...


In [4]:
# Drop irrelevant variables
df_earnings = df_earnings.drop(['periodicity','seasonadj','geo_level','ind_level','ownercode','agegrp'
                 ,'race','ethnicity','firmage','EarnBeg','EarnHirAS','EarnHirNS','EarnSepS'
                 ,'sEarnBeg','sEarnHirAS','sEarnHirNS','sEarnS','sEarnSepS','version'],axis=1)

In [5]:
# Clean data
earnings_clean = df_earnings[:][df_earnings['EarnS'].isnull()==False] # Delete missing data
df_earnings.isnull().any() # Check if there is any missing data on rows and columns
df_earnings['geography'] = df_earnings['geography'].apply(int) # Convert the column of geography to integer

In [6]:
# Match the States name with States' geography number
# Read in the dataset containing names of the States
state_df = gpd.read_file("https://datascience.quantecon.org/assets/data/cb_2016_us_state_5m.zip")
state_df = state_df.rename(columns={'GEOID':'geography'})
state_df['geography'] = state_df['geography'].apply(int)
state_df = state_df.sort_values(by='geography')
state_df.isnull().any() 

STATEFP      False
STATENS      False
AFFGEOID     False
geography    False
STUSPS       False
NAME         False
LSAD         False
ALAND        False
AWATER       False
geometry     False
dtype: bool

In [7]:
# Create a master data file by merging the above two datasets
df_master = pd.merge(earnings_clean,state_df, on="geography")
df_master = df_master.drop(['STATEFP','STATENS','AFFGEOID','STUSPS','LSAD','ALAND','AWATER'],axis=1)
df_master['geography'] = df_master['geography'].apply(int)
df_master = pd.merge(df_master,label_industry, on="industry")
df_master = df_master.rename(columns={'label':'industry_name'})
df_master = pd.merge(df_master,label_education, on="education")
df_master = df_master.rename(columns={'label':'degree'})
df_master.head()

,geography,industry,sex,education,firmsize,year,quarter,EarnS,NAME,geometry,industry_name,ind_level,degree
0,30,11,1,E1,1,2012,1,2268.0,Montana,"POLYGON ((-116.04915 48.48125, -116.04916 48.5...","Agriculture, Forestry, Fishing and Hunting",2,Less than high school
1,30,11,2,E1,1,2012,1,1395.0,Montana,"POLYGON ((-116.04915 48.48125, -116.04916 48.5...","Agriculture, Forestry, Fishing and Hunting",2,Less than high school
2,30,11,1,E1,2,2012,1,2923.0,Montana,"POLYGON ((-116.04915 48.48125, -116.04916 48.5...","Agriculture, Forestry, Fishing and Hunting",2,Less than high school
3,30,11,2,E1,2,2012,1,1390.0,Montana,"POLYGON ((-116.04915 48.48125, -116.04916 48.5...","Agriculture, Forestry, Fishing and Hunting",2,Less than high school
4,30,11,1,E1,3,2012,1,3025.0,Montana,"POLYGON ((-116.04915 48.48125, -116.04916 48.5...","Agriculture, Forestry, Fishing and Hunting",2,Less than high school


In [9]:
# subsample 1 -draw map
df_geograph = df_master.groupby(['NAME','year','quarter'],as_index=False)['EarnS'].agg('mean').rename(columns={'EarnS':'avgEarnS'})
df_geograph.head()

,NAME,year,quarter,avgEarnS
0,Alabama,2012,1,3050.369149
1,Alabama,2012,2,3078.322375
2,Alabama,2012,3,3054.218085
3,Alabama,2012,4,3228.426299
4,Alabama,2013,1,3108.977660


In [10]:
# subsample 2 -draw heatmap by industry and geography
df_heatmap_1 = df_master.groupby(['NAME','industry_name'],as_index=False)['EarnS'].agg('mean').rename(columns={'EarnS':'avgEarnS'})
df_heatmap_1.head()

,NAME,industry_name,avgEarnS
0,Alabama,Accommodation and Food Services,1709.964000
1,Alabama,Administrative and Support and Waste Managemen...,2575.409000
2,Alabama,"Agriculture, Forestry, Fishing and Hunting",3154.130898
3,Alabama,"Arts, Entertainment, and Recreation",1881.641000
4,Alabama,Construction,4050.942000


In [11]:
# subsample 3 -draw heatmap by industry and degree
df_heatmap_2 = df_master.groupby(['degree','industry_name'],as_index=False)['EarnS'].agg('mean').rename(columns={'EarnS':'avgEarnS'})
df_heatmap_2.head()

,degree,industry_name,avgEarnS
0,Bachelor's degree or advanced degree,Accommodation and Food Services,2393.601561
1,Bachelor's degree or advanced degree,Administrative and Support and Waste Managemen...,4752.790836
2,Bachelor's degree or advanced degree,"Agriculture, Forestry, Fishing and Hunting",4279.834435
3,Bachelor's degree or advanced degree,"Arts, Entertainment, and Recreation",4099.492722
4,Bachelor's degree or advanced degree,Construction,6140.558370


## Cleaning the second dataset: PSEO of New York

In [78]:
# Read in the dataset
pseof_ny_0 = pd.read_excel("pseo_ny.xlsx")

In [79]:
# Drop irrelevant variables
pseof_ny_1 = pseof_ny_0.drop(['agg_level_pseo', 'label_agg_level_pseo', 'inst_level', 'label_inst_level',
                            'institution', 'label_institution', 'grad_cohort', 'label_grad_cohort', 
                            'grad_cohort_years', 'label_grad_cohort_years', 'status_y1_grads_emp', 'status_y1_grads_emp_instate', 
                            'status_y5_grads_emp', 'status_y5_grads_emp_instate', 'status_y10_grads_emp', 'status_y10_grads_emp_instate', 
                            'status_y1_grads_nme', 'status_y5_grads_nme', 'status_y10_grads_nme'],axis=1)

In [80]:
# The college degree number is in the column "cipcode", the industry number is in the column "industry"
pseof_ny_1 = pseof_ny_1[(pseof_ny_1['industry'] != "00") & (pseof_ny_1['geo_level'] == "N") & (pseof_ny_1['cipcode'] != 0)].reset_index()

In [81]:
pseof_ny_1.columns

Index(['index', 'Degree\nAward\nLevel', 'label_degree_level', 'cip_level',
       'label_cip_level', 'cipcode', 'label_cipcode', 'geo_level',
       'label_geo_level', 'geography', 'label_geography', 'ind_level',
       'label_ind_level', 'industry', 'label_industry', 'y1_grads_emp',
       'y1_grads_emp_instate', 'y5_grads_emp', 'y5_grads_emp_instate',
       'y10_grads_emp', 'y10_grads_emp_instate', 'y1_grads_nme',
       'y5_grads_nme', 'y10_grads_nme'],
      dtype='object')

In [82]:
pseof_ny_2 = pseof_ny_1.replace({'.':0})
pseof_ny_2.head()

,index,Degree\nAward\nLevel,label_degree_level,cip_level,label_cip_level,cipcode,label_cipcode,geo_level,label_geo_level,geography,...,label_industry,y1_grads_emp,y1_grads_emp_instate,y5_grads_emp,y5_grads_emp_instate,y10_grads_emp,y10_grads_emp_instate,y1_grads_nme,y5_grads_nme,y10_grads_nme
0,14390,5,Baccalaureate,2,2-Digit CIP\nFamily,3,Natural Resources and\nConservation,N,National (50\nStates + DC),0,...,"Agriculture, Forestry,\nFishing and Hunting",0,0,0,0,0,0,0,0,0
1,14391,5,Baccalaureate,2,2-Digit CIP\nFamily,3,Natural Resources and\nConservation,N,National (50\nStates + DC),0,...,"Mining, Quarrying, and\nOil and Gas Extraction",0,0,0,0,0,0,0,0,0
2,14392,5,Baccalaureate,2,2-Digit CIP\nFamily,3,Natural Resources and\nConservation,N,National (50\nStates + DC),0,...,Utilities,0,0,0,0,0,0,0,0,0
3,14393,5,Baccalaureate,2,2-Digit CIP\nFamily,3,Natural Resources and\nConservation,N,National (50\nStates + DC),0,...,Construction,0,0,0,0,0,0,0,0,0
4,14394,5,Baccalaureate,2,2-Digit CIP\nFamily,3,Natural Resources and\nConservation,N,National (50\nStates + DC),0,...,Manufacturing,0,0,0,0,0,0,0,0,0


In [83]:
# Get the degree majors and their corresponding numbers
cip = pseof_ny_2[['cipcode','label_cipcode']]
cip = cip.drop_duplicates().reset_index().drop(['index'],axis=1)
# Get the industries and their corresponding numbers
ind = pseof_ny_2[['industry','label_industry']]
ind = ind.drop_duplicates().reset_index().drop(['index'],axis=1)

In [85]:
# Groupby the dataset by college major number and industry number, then get the sum of the number of students in each category
pseof_ny_3 = pseof_ny_2.groupby(['cipcode','industry'])[['y1_grads_emp','y1_grads_emp_instate','y5_grads_emp','y5_grads_emp_instate',
                                                         'y10_grads_emp','y10_grads_emp_instate','y1_grads_nme','y5_grads_nme',
                                                         'y10_grads_nme']].sum()

In [86]:
pseo_ny_4 = pseof_ny_3.reset_index()

In [87]:
pseo_ny_4.head()

,cipcode,industry,y1_grads_emp,y1_grads_emp_instate,y5_grads_emp,y5_grads_emp_instate,y10_grads_emp,y10_grads_emp_instate,y1_grads_nme,y5_grads_nme,y10_grads_nme
0,1,11,417,384,249,214,106,82,0,0,0
1,1,21,2,2,7,5,3,2,0,0,0
2,1,22,7,7,4,3,3,3,0,0,0
3,1,23,132,92,117,74,58,37,0,0,0
4,1,31-33,212,109,203,103,124,54,0,0,0


In [88]:
# Match major and industry numbers with their names
pseo_ny_5 = pd.merge(pseo_ny_4,cip,on='cipcode')
pseo_ny = pd.merge(pseo_ny_5,ind,on='industry')

In [89]:
pseo_ny.head()

,cipcode,industry,y1_grads_emp,y1_grads_emp_instate,y5_grads_emp,y5_grads_emp_instate,y10_grads_emp,y10_grads_emp_instate,y1_grads_nme,y5_grads_nme,y10_grads_nme,label_cipcode,label_industry
0,1,11,417,384,249,214,106,82,0,0,0,Agricultural/Animal/Plant/Veterinary\nScience ...,"Agriculture, Forestry,\nFishing and Hunting"
1,3,11,76,58,45,28,26,18,0,0,0,Natural Resources and\nConservation,"Agriculture, Forestry,\nFishing and Hunting"
2,4,11,5,0,2,0,6,3,0,0,0,Architecture and Related Services,"Agriculture, Forestry,\nFishing and Hunting"
3,5,11,22,8,9,1,9,8,0,0,0,"Area, Ethnic, Cultural, Gender, and\nGroup Stu...","Agriculture, Forestry,\nFishing and Hunting"
4,9,11,42,20,22,14,18,14,0,0,0,"Communication, Journalism, and\nRelated Programs","Agriculture, Forestry,\nFishing and Hunting"
